In [134]:
# Input parameters here
salary = 100000
principle = 50000
down_payment_amount = 25000
salary_increase = 1.05

# set variables
repayment_rate = 0.09
loan_interest_rate = 0.073
monthly_loan_interest_rate = loan_interest_rate/12


In [135]:
def monthly_repayment_calc(salary, salary_increase):
    """
    Function to calculate the increase in monthly repayment as salary increases over time.
    Return monthly repayments amount.
    """
    # I think this is the amount before you start paying repayments (i.e. that 27000 is exempt). Figured it out from my repayments
    repayment_threshold = 27000
    salary = salary * salary_increase
    loan_salary = salary - repayment_threshold
    monthly_salary = loan_salary/12
    monthly_repayment = monthly_salary * repayment_rate
    return monthly_repayment

In [136]:
def months_to_pay_off(principle, down_payment_amount, down_payment, monthly_loan_interest_rate, salary, salary_increase):
    """
    Calculate the number of months it will take to pay off the loan.
    If there is a down payment then take it off the principle amount.
    Every 12 months run the monthly_repayment_calc to increase the salary. 
    """
    months = 0
    previous_principle = principle
    loop_counter = 0

    if down_payment:
        principle = principle - down_payment_amount
    else:
        principle = principle
    
    monthly_repayment = monthly_repayment_calc(salary=salary, salary_increase=salary_increase)
    
    while principle > 0:
        principle_interest = principle * monthly_loan_interest_rate
        principle = (principle + principle_interest) - monthly_repayment
        months += 1
        loop_counter += 1
        
        if loop_counter > 36 and principle > previous_principle:
            # check if the principle has increased after 3 years. If it has, won't pay off the loan with current parameters and the function hangs
            raise ValueError("The principle is trending upwards after 36 months. You won't pay off your loan early. Adjust your parameters")
        
        # every year assume the salary increases by the salary increase percentage
        if months % 12 == 0:
            salary = salary * salary_increase 
            monthly_repayment = monthly_repayment_calc(salary=salary, salary_increase=salary_increase)

    return months

In [137]:
def total_payments_made(principle, down_payment_amount, salary, salary_increase, monthly_loan_interest_rate):
    """Calculate the total payments needed to pay off the loan"""
    balance = principle - down_payment_amount

    months = 0
    total_paid = 0
    monthly_repayment = monthly_repayment_calc(salary=salary, salary_increase=salary_increase)

    while balance > 0:
        interest = balance * monthly_loan_interest_rate
        balance = balance + interest - monthly_repayment
        total_paid += monthly_repayment
        months += 1
        monthly_repayment = monthly_repayment_calc(salary=salary, salary_increase=salary_increase)

    return round(total_paid + down_payment_amount)

In [138]:
def compound_interest_calculator(principle, return_rate, compounding_periods, years, monthly_deposit):
    """Compound interest calcuator (generated by chatGPT)"""
    P = principle 
    r = return_rate
    n = compounding_periods
    t = years
    D = monthly_deposit
    
    output = P * (1 + r /n ) ** (n * t) + D * (((1 + r/n) ** (n * t) - 1) / (r / n))
    return output

In [139]:
def loan_payment_calculator(principle, down_payment_amount, down_payment: bool, monthly_loan_interest_rate, salary, salary_increase):
    """ 
    Calculate loan repaying metrics:
    - months to pay back loan
    - total cost of paying back loan 
    - investment gain
    """
    # total months it will take to pay off the loan if we don't take any action
    total_months_no_action = months_to_pay_off(principle=principle, down_payment=0, down_payment_amount=down_payment_amount, monthly_loan_interest_rate=monthly_loan_interest_rate, salary=salary, salary_increase=salary_increase)

    # calculate months and total payments it'll take to pay off loan
    if down_payment: 
        months = months_to_pay_off(principle=principle, down_payment_amount=down_payment_amount, down_payment=down_payment, monthly_loan_interest_rate=monthly_loan_interest_rate, salary=salary, salary_increase=salary_increase)
        total_payment = total_payments_made(principle=principle, down_payment_amount=down_payment_amount, salary=salary, salary_increase=salary_increase, monthly_loan_interest_rate=monthly_loan_interest_rate)

    else:
        months = months_to_pay_off(principle=principle, down_payment=0, down_payment_amount=down_payment_amount, monthly_loan_interest_rate=monthly_loan_interest_rate, salary=salary, salary_increase=salary_increase)
        total_payment = total_payments_made(principle=principle, down_payment_amount=0, salary=salary, salary_increase=salary_increase, monthly_loan_interest_rate=monthly_loan_interest_rate)

    # the maximum theoritical months to pay off the loan
    investment_years_no_action = total_months_no_action/12
    
    # the number of months we have to invest (used when we make a down payment to work out time we'd have left to invest once loan is paid off)
    investment_months = total_months_no_action - months
    investment_years = investment_months/12

    monthly_repayment = monthly_repayment_calc(salary=salary, salary_increase=salary_increase)

    if not down_payment:
        # if no down payment made, then use that down payment and invest it. Assumes no extra monthly deposit as it's being used to pay off the loan in monthly payments
        investment_gains = compound_interest_calculator(principle=down_payment_amount, return_rate=0.08, compounding_periods=12, years=investment_years_no_action, monthly_deposit=0) 

    else:
        # if a down payment is made, then initial investment is 0, but we add the amount we would have paid to the loan into stocks instead
        investment_gains = compound_interest_calculator(principle=0, return_rate=0.08, compounding_periods=12, years=investment_years, monthly_deposit=monthly_repayment) 


    return months, total_payment, round(investment_gains)

In [140]:
# NO DOWN PAYMENT MADE

months, total_payment, investment_gains = loan_payment_calculator(
    principle=principle,
    down_payment_amount=down_payment_amount,
    down_payment=False,
    monthly_loan_interest_rate=monthly_loan_interest_rate,
    salary=salary, 
    salary_increase=salary_increase,
)


print("NO DOWN PAYMENT MADE")
print(f"Years to pay off loan: {months/12}")
print(f"Total cost of paying off loan: £{total_payment}")
print(f"Gains from investment from leaving {down_payment_amount} in stocks: £{investment_gains}")
print(f"Total gain: £{investment_gains - total_payment}")

NO DOWN PAYMENT MADE
Years to pay off loan: 7.666666666666667
Total cost of paying off loan: £70785
Gains from investment from leaving 25000 in stocks: £46071
Total gain: £-24714


In [141]:
# DOWN PAYMENT MADE

months, total_payment, investment_gains = loan_payment_calculator(
    principle=principle,
    down_payment_amount=down_payment_amount,
    down_payment=True,
    monthly_loan_interest_rate=monthly_loan_interest_rate,
    salary=salary, 
    salary_increase=salary_increase,
)


print(f"DOWN PAYMENT OF £{round(down_payment_amount)} MADE")
print(f"Years to pay off loan: {months/12}")
print(f"Total cost of paying off loan: £{total_payment}")
print(f"Gains from investment (extra years investing using money that would have been spent on repayments): £{investment_gains}")
print(f"Total gain: £{investment_gains - total_payment}")

DOWN PAYMENT OF £25000 MADE
Years to pay off loan: 3.75
Total cost of paying off loan: £54250
Gains from investment (extra years investing using money that would have been spent on repayments): £32165
Total gain: £-22085
